In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
data = pd.read_csv('biddings.csv')
dfo = pd.DataFrame(data)


In [16]:
dfo = dfo.sort_values(by=['convert'])
dfo = dfo.reset_index(drop=True)
df1 = dfo.drop(dfo.index[0:997500])


In [17]:
df = df1.reset_index(drop=True)
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,convert
2495,-0.02,2.33,-3.71,-0.10,-0.94,0.40,-7.08,-1.26,0.82,-0.46,...,-0.09,0.70,0.31,0.02,0.49,0.06,0.09,0.45,-0.12,1
2496,0.01,1.88,-1.23,-0.70,0.98,0.22,-1.39,0.05,-0.16,-0.58,...,0.15,0.22,-0.64,-0.10,-0.03,0.35,-0.02,0.56,-0.17,1
2497,-0.01,-1.18,1.99,-0.63,0.79,0.28,0.13,0.50,-0.32,0.96,...,-0.06,-1.41,0.24,0.83,-0.10,0.47,-0.17,0.42,-0.13,1
2498,0.09,-8.92,-4.29,0.38,-1.54,-1.63,2.09,-2.65,2.59,-4.74,...,0.02,0.13,-0.07,0.44,-0.01,0.05,0.09,-0.27,0.03,1
2499,-0.01,-0.76,-4.40,-0.19,-0.76,2.45,0.92,-0.50,-0.18,0.01,...,0.01,-1.22,0.24,0.22,0.31,0.10,0.13,-0.17,0.08,1


In [18]:
#Convert column = shown ad was clicked: 1 = clicked ad
df.convert.value_counts()

1    1908
0     592
Name: convert, dtype: int64

### Severely underbalanced

Converted accounts for 1908/999999 = .191% of our data

We will have to oversample or undersample our target. Most likely undersample to minimize our noise

I will use RandomUnderSampler, NearMiss, and EditedNearestNeighbors
to underersample our target here first to get baselines

In [19]:
from sklearn.model_selection import train_test_split

x = df.drop(['convert'],axis=1) #training features
y = df.convert #target

#Need to set our train/test values before undersampling
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                  test_size = .10)

In [20]:
y_test.value_counts()

1    180
0     70
Name: convert, dtype: int64

In [21]:
#First model using our undersampled data (RandomUnderSampler)
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

clf = LinearSVC()
clf.fit(x_train, y_train) 

print('Training accuracy LinearSVC is: {}'.format(clf.score(x_train, y_train)))
print('Test accuracy LinearSVC is: {}'.format(clf.score(x_test, y_test)))

scores = cross_val_score(clf, x_train, y_train, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

y_pred = clf.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy LinearSVC is: 0.7768888888888889
Test accuracy LinearSVC is: 0.728
Cross Val LinearSVC Accuracy: 0.76 (+/- 0.02)
              precision    recall  f1-score   support

           0       0.58      0.10      0.17        70
           1       0.74      0.97      0.84       180

   micro avg       0.73      0.73      0.73       250
   macro avg       0.66      0.54      0.50       250
weighted avg       0.69      0.73      0.65       250



In [30]:
from sklearn import ensemble

params = {'n_estimators': 100,
          'max_features': 'sqrt',
          'max_depth': 6}

rfc = ensemble.RandomForestClassifier(**params)
rfc.fit(x_train, y_train)

print('Training accuracy normal RandomForest is: {}'.format(rfc.score(x_train, y_train)))
print('Test accuracy normal RandomForest is: {}'.format(rfc.score(x_test, y_test)))

scores = cross_val_score(rfc, x_train, y_train, cv=5)
print("Cross Val LinearSVC Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

y_pred = rfc.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy normal RandomForest is: 0.7871111111111111
Test accuracy normal RandomForest is: 0.74
Cross Val LinearSVC Accuracy: 0.77 (+/- 0.00)
              precision    recall  f1-score   support

           0       1.00      0.07      0.13        70
           1       0.73      1.00      0.85       180

   micro avg       0.74      0.74      0.74       250
   macro avg       0.87      0.54      0.49       250
weighted avg       0.81      0.74      0.65       250



In [23]:
params = {'n_estimators': 250,
          'max_features': 'sqrt',
          'max_depth': 8}

clf1 = ensemble.GradientBoostingClassifier(**params)
clf1.fit(x_train, y_train)

print('Training accuracy normal GradientBoosting is: {}'.format(clf1.score(x_train, y_train)))
print('Test accuracy normal GradientBoosting is: {}'.format(clf1.score(x_test, y_test)))

scores = cross_val_score(clf1, x_train, y_train, cv=5)
print("CrossVal GradientBoosting Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(),scores.std() * 2))

y_pred = clf1.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy normal GradientBoosting is: 0.9946666666666667
Test accuracy normal GradientBoosting is: 0.724
CrossVal GradientBoosting Accuracy: 0.76 (+/- 0.02)
              precision    recall  f1-score   support

           0       0.52      0.19      0.27        70
           1       0.75      0.93      0.83       180

   micro avg       0.72      0.72      0.72       250
   macro avg       0.63      0.56      0.55       250
weighted avg       0.68      0.72      0.67       250



In [24]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score

y_pred = clf.predict(x_test) #predict y based on x_test
balanced_accuracy_score(y_test, y_pred)

0.5361111111111111

In [25]:
f1_score(y_test, y_pred)

0.8373205741626794

In [26]:
%timeit clf.predict(df.iloc[5:6, 0:88] )

1.51 ms ± 473 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(x_train, y_train)

print('Training accuracy Logistic is: {}'.format(log.score(x_train, y_train)))
print('Test accuracy Logistic is: {}'.format(log.score(x_test, y_test)))

scores = cross_val_score(log, x_train, y_train, cv=5)
print("Cross Val Logistic Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

y_pred = log.predict(x_test) #predict y based on x_test
print(classification_report(y_test, y_pred)) #true value vs predicted value

Training accuracy Logistic is: 0.7755555555555556
Test accuracy Logistic is: 0.736
Cross Val Logistic Accuracy: 0.76 (+/- 0.02)
              precision    recall  f1-score   support

           0       0.62      0.14      0.23        70
           1       0.74      0.97      0.84       180

   micro avg       0.74      0.74      0.74       250
   macro avg       0.68      0.55      0.54       250
weighted avg       0.71      0.74      0.67       250



In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [100, 250],
    'max_features': ['sqrt'],
    'max_depth' : [5,6,7,8]
}

CV_clf = GridSearchCV(estimator=clf1, param_grid=param_grid, cv= 4)
CV_clf.fit(x_train, y_train)

CV_clf.best_params_

{'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 100}